In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
#from sklearn import preprocessing

In [11]:
data = pd.read_csv('datatest2.txt')
data.head(5)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9752 entries, 1 to 9752
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           9752 non-null   object 
 1   Temperature    9752 non-null   float64
 2   Humidity       9752 non-null   float64
 3   Light          9752 non-null   float64
 4   CO2            9752 non-null   float64
 5   HumidityRatio  9752 non-null   float64
 6   Occupancy      9752 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 609.5+ KB


In [12]:
def toOrdinalDate(column):
    arr=[]
    for date in column:
        time=pd.Timestamp(date)
        time=time.toordinal()
        arr.append(time)
    return pd.DataFrame(np.array(arr), index=data.index)
data['date'] = toOrdinalDate(data['date'].to_numpy())

In [15]:
def rescaling(dfColumn):
    feature = dfColumn.to_numpy().reshape(-1, 1)
    minmax_scale = MinMaxScaler(feature_range=(0, 1))
    scaled_feature = minmax_scale.fit_transform(feature)
    return pd.Series(scaled_feature.reshape(-1), index=data.index)
for column in data:
    data[column] = rescaling(data[column])

In [22]:
data.head(5)

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
1,0.0,0.462168,0.525565,0.276618,0.342373,0.700223,1.0
2,0.0,0.468303,0.518004,0.276618,0.323736,0.695239,1.0
3,0.0,0.463701,0.524950,0.274510,0.326092,0.700447,1.0
4,0.0,0.463701,0.524950,0.277672,0.329704,0.700447,1.0
5,0.0,0.468303,0.525565,0.276618,0.327296,0.703947,1.0
